In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/JCasado/desktop/IRONHACK/ironhack_service_account_big_query.json"

In [2]:
from google.cloud import bigquery
client = bigquery.Client()

#### Step 1: Calculate the royalties of each sales for each author

In [ ]:
royalties_per_sale = '''SELECT 

    ptitles.title_id AS TITLE_ID,
    pauthors.au_lname AS LAST_NAME,
    pauthors.au_fname AS FIRST_NAME,
    ptitles.price AS PRICE,
    ptitles.advance AS ADVANCE,
    ptitles.royalty AS ROYALTY,
    psales.qty AS QTY,
    ptauthor.au_id AS AU_ID
    ptitles.title AS TITLE, 
    ptauthor.royaltyper AS ROYALTYPER
    (ptitles.price*(ptitles.royalty/100)*psales.qty*(ptauthor.royalteper/100)) AS ROYALTIES
    
FROM 
    `ironhack-data-analytics-265219.publications.titleauthor` AS ptauthor

JOIN
    publications.authors as pauthors
ON
    ptauthor.au_id = ptauthors.au_id

JOIN    
    publications.titles AS ptitles
ON
    ptauthor.title_id = ptitles.title_id

JOIN    
    publications.sales AS psales
ON   
    ptauthor.title_id = psales.title_id

GROUP BY
    ptitles.title_id,
    ptitles.advance,
    ptitles.price,
    ptitles.royalty,
    ptauthor.au_id,
    pauthors.au_fname,
    pauthors.au_lname,
    psales.qty,
    ptauthor.royaltyper
    
ORDER BY
    ptitles.titles_id
    
LIMIT
    5
    
'''


In [ ]:
query_job = client.query(query=royalties_per_sale)
query_job.to_dataframe()

#### Step 2: Aggregate the total royalties for each title for each author

In [ ]:
royalties_per_title = '''

SELECT
    royalties_per_sale.title_id,
    royalties_per_sale.au_fname,
    royalties_per_sale.au_lname,
    royalties_per_sale.au_id,
    royalties_per_sale.advance,
    SUM(royalties) AS ROYALTIES
    
FROM
    (SELECT
    ptitles.title_id AS TITLE_ID,
    pauthors.au_lname AS LAST_NAME,
    pauthors.au_fname AS FIRST_NAME,
    ptitles.price AS PRICE,
    ptitles.advance AS ADVANCE,
    ptitles.royalty AS ROYALTY,
    psales.qty AS QTY,
    ptauthor.au_id AS AU_ID
    ptitles.title AS TITLE, 
    ptauthor.royaltyper AS ROYALTYPER
    (ptitles.price*(ptitles.royalty/100)*psales.qty*(ptauthor.royalteper/100)) AS ROYALTIES
    
    )

    FROM 
        `ironhack-data-analytics-265219.publications.titleauthor` as ptauthor
    
    JOIN
        publications.authors as pauthors
    ON
        ptauthor.au_id = pauthors.au_id
    
    JOIN
        publications.titles as ptitles
    ON
        ptauthor.title_id = ptitles.title_id
        
    JOIN
        publications.sales as psales
    ON
        ptauthor.title_id = psales.title_id
    
    GROUP BY
        ptitles.title_id,
        ptitles.price,
        ptitles.advance,
        ptitles.royalty,
        psales.qty,
        ptauthor.au_id,
        pauthors.au_fname,
        pauthors.au_lname,
        ptauthor.royaltyper
    
    ORDER BY
        ptitles.title_id AS royalties_per_sale

GROUP BY
    royalties_per_sale.title_id,
    royalties_per_sale.au_fname,
    royalties_per_sale.au_lname,
    royalties_per_sale.au_id,
    royalties_per_sale.advance
    
LIMIT
    5
'''


In [ ]:
query_job = client.query(query=royalties_per_title)
query_job.to_dataframe()

#### Step 3: Calculate the total profits of each author

In [ ]:
#no time! i will do it next days

#### Challenge 2

In [ ]:
most_profiting_authors = '''

CREATE TABLE 
    `ironhack-data-analytics-265219.publications.most_profiting_authors` (
    
    au_id STRING,
    au_fname STRING,
    au_lname STRING,
    total_profits NUMERIC)

In [ ]:
sql= '''

INSERT INTO `ironhack-data-analytics-265219.publications.most_profiting_authors`

VALUES
('899-46-2035', 'Anne', 'Ringer', 39178.132),
('722-51-5454', 'Michel', 'DeFrance', 30021.528),
('213-46-8915', 'Marjorie','Green', 20162.110)
'''